# This is analysis of data from the Stanford COVID-19 incidence paper


### Computational Environment Setup

_We are using almond for Scala support in Jupyter, and using SoS to enable simultaneous use of multiple kernels.  The kernels first need to be set up, which happens below._

#### Jupyter SoS Environment Setup (Python)

In [1]:
import json
import re
import ipykernel
import requests

from requests.compat import urljoin
from notebook.notebookapp import list_running_servers

def get_notebook_name():
    kernel_id = re.search('kernel-(.*).json', ipykernel.connect.get_connection_file()).group(1)
    servers = list_running_servers()
    for ss in servers:
        response = requests.get(urljoin(ss['url'], 'api/sessions'), params={'token': ss.get('token', '')})
        for nn in json.loads(response.text):
            if nn['kernel']['id'] == kernel_id:
                return nn['notebook']['path'][:-6]

#### Scala Environment Setup

In [2]:
interp.repositories () ++= Seq(coursier.maven.MavenRepository("https://oss.sonatype.org/content/repositories/snapshots"))

In [3]:
import $ivy.`com.github.ichoran::kse:0.10-SNAPSHOT`

import $ivy.$                                      


In [4]:
%expand {{{ }}}
import kse.flow._, kse.coll._, kse.coll.packed._
import kse.maths._, kse.maths.stats._, kse.maths.hashing._, kse.maths.stochastic._, kse.maths.optimization._
import kse.jsonal._, JsonConverters._
import kse.eio._, kse.alien._
import kse.visual._,kse.visual.chart._
val the_notebook_name = "{{{get_notebook_name()}}}"

def view(v: Vector[Indent]): Unit =
  display(almond.interpreter.api.DisplayData.svg(v.mkString("\n")))
def see(size: Vc, stuff: InSvg*): Unit = {
  val lines = kse.visual.chart.svg(size, stuff: _*)
  lines.map(_.toString).toFile("out.svg".file)
  view(lines)
}
def see(stuff: InSvg*): Unit = see(Vc(600, 600), stuff: _*)

def printSee(svgs: InSvg*) = {
  svgHtml(Vc(600, 600), svgs: _*).toFile("out.html".file)
  kse.visual.chart.svg(Vc(600, 600), svgs: _*).map(_.toString).toFile("out.svg".file)
  see(svgs: _*)
}


import kse.flow._, kse.coll._, kse.coll.packed._

import kse.maths._, kse.maths.stats._, kse.maths.hashing._, kse.maths.stochastic._, kse.maths.optimization._

import kse.jsonal._, JsonConverters._

import kse.eio._, kse.alien._

import kse.visual._,kse.visual.chart._

the_notebook_name: String = "Stanford-COVID19-Incidence"
defined function view
defined function see
defined function see
defined function printSee

## Analysis (no notes--just exploring)

In [5]:
def logC(n: Int, k: Int): Double = 
  if (k > n/2) logC(n, n-k)
  else {
    var sum = 0.0
    var i = 1
    while (i <= k) {
      sum += ((n+1-i).toDouble / i.toDouble).log
      i += 1
    }
    sum
  }
  
def logBin(n: Int, k: Int, rate: Double): Double =
  if (rate <= 0) { if (k == 0) 0 else Double.NegativeInfinity }
  else if (rate >= 1) { if (k == n) 0 else Double.NegativeInfinity }
  else logC(n, k) + k * rate.log + (n-k) * (1-rate).log

def binom(n: Int, k: Int, rate: Double) = logBin(n, k, rate).exp

def pcontrol(n: Int, k: Int, s: Double): Double = binom(n, k, s)

def ptest(n: Int, nq: Int, k: Int, r: Double, s: Double): Double = {
  var i = 0
  val iN = nq min k
  var p = 0.0
  while (i <= iN) {
    p += binom(nq, i, r)*binom(n-nq, k-i, s)
    i += 1
  }
  p
}

def ptotal(nc: Int, kc: Int)(n: Int, k: Int)(nq: Int, r: Double, s: Double) =
  pcontrol(nc, kc, s) * ptest(n, nq, k, r, s)
  
ptotal(401, 2)(3330, 50)(44, 102.toDouble/133, 0.005)

/*
// Estimate spread from false negatives
val pvals = Array.tabulate(111, 330){ (nq, se4) =>
  val s = se4*1e-4
  var p = 0.0
  var i = 82
  while (i <= 120) {
    p += binom(133, i, 102.toDouble/133) *
         ptotal(401, 2)(3330, 50)(nq, i.toDouble/133, s)
    i += 1
  }
  p
}
*/

// Assume false negatives are exactly as measured
val pvals = Array.tabulate(111, 330){ (nq, se4) =>
  val s = se4 * 1e-4
  ptotal(401, 2)(3330, 50)(nq, 102.toDouble/133, s)
}

defined function logC
defined function logBin
defined function binom
defined function pcontrol
defined function ptest
defined function ptotal
res3_6: Double = 0.022051100222845253
pvals: Array[Array[Double]] = Array(
  Array(
    0.0,
    1.6709586480083563E-92,
    5.2086426973137425E-77,
    5.171936015188717E-68,
    1.1236590797023486E-61,
    8.513465292526325E-57,
    7.720871040589737E-53,
    1.6181489539863814E-49,
    1.1606542793645849E-46,
    3.6704657701035764E-44,
    6.083896469559105E-42,
    5.979385598196901E-40,
    3.816812556319825E-38,
    1.6957172829995473E-36,
    5.533043077346571E-35,
    1.3837024631496106E-33,
    2.744804155210699E-32,
    4.441965369531576E-31,
    6.00267019064401E-30,
    6.906983065289879E-29,
    6.879736463326614E-28,
    6.016032841718121E-27,
    4.674821948461299E-26,
    3.262089675611383E-25,
    2.0629071994423557E-24,
    1.191807786589865E-23,
    6.335114216532728E-23,
    3.117845690201876E-22,
    1.4286931825388381E-21,


In [104]:
val pmax = pvals.map(_.max).max
val image = Bitmap.fillRgba(111, 330){ (nq, se4) =>
  Spectrum.Fire(((pvals(nq)(se4)/pmax).log10/2+1).toFloat max 0)
}
val maxColor = Spectrum.Fire(1)
val fifthColor = Spectrum.Fire((1 + 0.2.log10/2).toFloat)
val twentyfifthColor = Spectrum.Fire((1 + 0.04.log10/2).toFloat)
printSee(Space(
  Vc(0, 0), Vc(3.05f, 3.05f),
  Vc(100, 100), Vc(400, 400),
  12, 6, None,
  Stroke(Rgba.Black, 3f) + FontSize(18),
  Seq(
    Picture(Vc(0, 0), image, Some(Vc.from(1/33.3, 1/100.0)), Fill(Rgba.Black)),
    Marker.C(Vc(3, 3), 6f, Fill(maxColor)),
    Letters(Vc(2.8f, 3), "Max likelihood", 0, Fill(maxColor) ++ Font(18, Horizontal.Right, Vertical.Middle)),
    Marker.C(Vc(3, 2.8f), 6f, Fill(fifthColor)),
    Letters(Vc(2.8f, 2.8f), "1/5th likelihood", 0, Fill(fifthColor) ++ Font(18, Horizontal.Right, Vertical.Middle)),
    Marker.C(Vc(3, 2.6f), 6f, Fill(twentyfifthColor)),
    Letters(Vc(2.8f, 2.6f), "1/25th likelihood", 0, Fill(twentyfifthColor) ++ Font(18, Horizontal.Right, Vertical.Middle)),
  ),
  Some(Titling("Likelihood given parameters (combined, noisy r)", "Sample incidence (%)", "False positive rate (%)", ylegendGap = -0.1f))
))


pmax: Double = 0.022425314838301253
image: Bitmap = kse.visual.Bitmap@36c83543
maxColor: Rgba = #FFFFFFFF
fifthColor: Rgba = #FFDA00FF
twentyfifthColor: Rgba = #FF2100FF

In [106]:
val psum = pvals.map(_.sum).sum
val pnorm = pvals.map(_.map(_ / psum))
val psort = pnorm.flatten.sorted.reverse
val pidx = psort.scan(0.0)(_ + _).indexWhere(_ > 0.95)
val plim = psort(pidx)
val imageT = Bitmap.fillRgba(111, 330){ (nq, se4) =>
  Spectrum.Fire( if (pnorm(nq)(se4) >= plim) 0.4f else 1 )
}
printSee(Space(
  Vc(0, 0), Vc(3.05f, 3.05f),
  Vc(100, 100), Vc(400, 400),
  12, 6, None,
  Stroke(Rgba.Black, 3f) + FontSize(18),
  Seq(Picture(Vc(0, 0), imageT, Some(Vc.from(1/33.3, 1/100.0)), Fill(Rgba.Black))),
  Some(Titling("95% likelihood region (combined)", "Sample incidence (%)", "False positive rate (%)", ylegendGap = -0.1f))
))


psum: Double = 30.62501863110763
pnorm: Array[Array[Double]] = Array(
  Array(
    0.0,
    5.456188184359391E-94,
    1.7007802542275086E-78,
    1.688794406131488E-69,
    3.6690886403608E-63,
    2.779905343103595E-58,
    2.521099214204949E-54,
    5.2837484720506667E-51,
    3.789889218828557E-48,
    1.198518706001788E-45,
    1.986577230480224E-43,
    1.9524512524290477E-41,
    1.2463053826334214E-39,
    5.537032657596844E-38,
    1.8067068444903194E-36,
    4.5182093758601104E-35,
    8.962620360408991E-34,
    1.4504367892921413E-32,
    1.960054380031216E-31,
    2.255340037009496E-30,
    2.2464431928013465E-29,
    1.964417692013184E-28,
    1.5264715443186075E-27,
    1.0651714909645433E-26,
    6.736019410440259E-26,
    3.8916148948209156E-25,
    2.0686074653021697E-24,
    1.0180714427500453E-23,
    4.6651177579615595E-23,
    2.0003558743761596E-22,
    8.06244832357846E-22,
    3.0669311562001578E-21,
    1.1051290442594225E-20,
    3.784783883216608E-20,
    1.2

In [64]:
val pvals2 = Array.tabulate(161, 430){ (nq, se4) =>
  val s = se4*1e-4
  ptotal(30, 0)(3330, 50)(nq, 25.toDouble/37, s)
}

pvals2: Array[Array[Double]] = Array(
  Array(
    0.0,
    2.1618103997801484E-89,
    1.7480117410427805E-74,
    8.004235863908233E-66,
    1.014973968467314E-59,
    5.106688452216736E-55,
    3.3371143250028164E-51,
    5.3316986377288034E-48,
    3.038110562822328E-45,
    7.876910858749392E-43,
    1.0973427314648205E-40,
    9.248557008395691E-39,
    5.1473659366950395E-37,
    2.021896727383581E-35,
    5.9026541468611964E-34,
    1.3342840372447626E-32,
    2.413847538436856E-31,
    3.5906057051297144E-30,
    4.4910038061976715E-29,
    4.812606007564869E-28,
    4.4891871150120585E-27,
    3.694757094441306E-26,
    2.714525907358483E-25,
    1.798357816265843E-24,
    1.0838178670769688E-23,
    5.988114893273551E-23,
    3.0537825732995166E-22,
    1.4461904676592844E-21,
    6.394270043330926E-21,
    2.6523263814249673E-20,
    1.0366043115996619E-19,
    3.8321606727928877E-19,
    1.344783005968918E-18,
    4.493969296624231E-18,
    1.4343142045426916E-17,
    4.38

In [73]:
val pmax2 = pvals2.map(_.max).max
val image2 = Bitmap.fillRgba(111, 330){ (nq, se4) =>
  Spectrum.Fire(((pvals2(nq)(se4)/pmax2).log10/2+1).toFloat max 0)
}
val maxColor = Spectrum.Fire(1)
val fifthColor = Spectrum.Fire((1 + 0.2.log10/2).toFloat)
val twentyfifthColor = Spectrum.Fire((1 + 0.04.log10/2).toFloat)
printSee(Space(
  Vc(0, 0), Vc(3.05f, 3.05f),
  Vc(100, 100), Vc(400, 400),
  12, 6, None,
  Stroke(Rgba.Black, 3f) + FontSize(18),
  Seq(
    Picture(Vc(0, 0), image2, Some(Vc.from(1/33.3, 1/100.0)), Fill(Rgba.Black)),
    Marker.C(Vc(3, 3), 6f, Fill(maxColor)),
    Letters(Vc(2.8f, 3), "Max likelihood", 0, Fill(maxColor) ++ Font(18, Horizontal.Right, Vertical.Middle)),
    Marker.C(Vc(3, 2.8f), 6f, Fill(fifthColor)),
    Letters(Vc(2.8f, 2.8f), "1/5th likelihood", 0, Fill(fifthColor) ++ Font(18, Horizontal.Right, Vertical.Middle)),
    Marker.C(Vc(3, 2.6f), 6f, Fill(twentyfifthColor)),
    Letters(Vc(2.8f, 2.6f), "1/25th likelihood", 0, Fill(twentyfifthColor) ++ Font(18, Horizontal.Right, Vertical.Middle)),
  ),
  Some(Titling("Likelihood given parameters (their controls)", "Sample incidence (%)", "False positive rate (%)", ylegendGap = -0.1f))
))


pmax2: Double = 0.0986717148071501
image2: Bitmap = kse.visual.Bitmap@40c4649d
maxColor: Rgba = #FFFFFFFF
fifthColor: Rgba = #FFDA00FF
twentyfifthColor: Rgba = #FF2100FF

In [74]:
val psum2 = pvals2.map(_.sum).sum
val pnorm2 = pvals2.map(_.map(_ / psum2))
val psort2 = pnorm2.flatten.sorted.reverse
val pidx2 = psort2.scan(0.0)(_ + _).indexWhere(_ > 0.95)
val plim2 = psort(pidx2)
val imageT2 = Bitmap.fillRgba(111, 330){ (nq, se4) =>
  Spectrum.Fire( if (pnorm2(nq)(se4) >= plim2) 0.4f else 1 )
}
printSee(Space(
  Vc(0, 0), Vc(3.05f, 3.05f),
  Vc(100, 100), Vc(400, 400),
  12, 6, None,
  Stroke(Rgba.Black, 3f) + FontSize(18),
  Seq(Picture(Vc(0, 0), imageT2, Some(Vc.from(1/33.3, 1/100.0)), Fill(Rgba.Black))),
  Some(Titling("95% likelihood region (their controls)", "Sample incidence (%)", "False positive rate (%)", ylegendGap = -0.1f))
))


psum2: Double = 183.50015524041447
pnorm2: Array[Array[Double]] = Array(
  Array(
    0.0,
    1.1780973138403246E-91,
    9.52594148355137E-77,
    4.361977707005974E-68,
    5.531188609282299E-62,
    2.7829341318682587E-57,
    1.8185893742872667E-53,
    2.905555382633561E-50,
    1.6556446826118042E-47,
    4.292590841914756E-45,
    5.980064322164341E-43,
    5.040081299265718E-41,
    2.8051016795877743E-39,
    1.101850145431525E-37,
    3.216702535824985E-36,
    7.271296503791169E-35,
    1.3154471369652689E-33,
    1.9567316989053487E-32,
    2.447411447861578E-31,
    2.6226713548332357E-30,
    2.4464214262546576E-29,
    2.0134899011941352E-28,
    1.4793044200981852E-27,
    9.80030678398995E-27,
    5.906359401478405E-26,
    3.2632751102734304E-25,
    1.664185280551166E-24,
    7.881140295301354E-24,
    3.4846128794569207E-23,
    1.445408249355428E-22,
    5.649065038890812E-22,
    2.088369172098054E-21,
    7.328511543802444E-21,
    2.4490275175715325E-20,
    7.

In [107]:
// Project combined results onto sample incidence axis to estimate 95% CI
val pflat = pnorm.map(_.sum)
val pflatnorm = { val x = pflat.sum; pflat.map(_ / x) }
val plim = pflatnorm.sorted.reverse.fn(x => x.scan(0.0)(_ + _) zip x).dropWhile(_._1 < 0.95).head._2
pflatnorm.indexWhere(_ > plim)
pflatnorm.length - pflatnorm.reverse.indexWhere(_ > plim)

pflat: Array[Double] = Array(
  0.004482618219857986,
  0.004748584350469923,
  0.005028084333129445,
  0.005321577972906422,
  0.005629514797007757,
  0.005952330070453559,
  0.006290440334542542,
  0.0066442384302835885,
  0.007014087967702287,
  0.007400317200817137,
  0.007803212267380038,
  0.008223009752072467,
  0.008659888532025503,
  0.009113960864280027,
  0.00958526267632761,
  0.010073743023285836,
  0.010579252678821854,
  0.011101531831729065,
  0.011640196866530285,
  0.012194726214692628,
  0.012764445273491113,
  0.013348510402582774,
  0.013945892024347184,
  0.014555356873566783,
  0.015175449465644607,
  0.015804472880891342,
  0.016440468996309812,
  0.0170811983364737,
  0.017724119762789287,
  0.018366370276372664,
  0.019004745275760255,
  0.019635679687630896,
  0.020255230478723373,
  0.020859061161892324,
  0.021442429030857947,
  0.02200017599926992,
  0.022526724082826553,
  0.02301607675156758,
...
pflatnorm: Array[Double] = Array(
  0.0044826182198579876,

In [68]:
// Project combined results onto sample incidence axis to estimate 95% CI
val pflat2 = pnorm2.map(_.sum)
val pflatnorm2 = { val x = pflat2.sum; pflat2.map(_ / x) }
val plim2 = pflatnorm2.sorted.reverse.fn(x => x.scan(0.0)(_ + _) zip x).dropWhile(_._1 < 0.95).head._2
pflatnorm2.indexWhere(_ > plim2)
pflatnorm2.length - pflatnorm2.reverse.indexWhere(_ > plim2)

pflat2: Array[Double] = Array(
  0.010320534052761338,
  0.010385991516232668,
  0.010451890097046942,
  0.010518232939266325,
  0.01058502321056589,
  0.010652264102412339,
  0.01071995883026093,
  0.010788110633734709,
  0.01085672277682695,
  0.010925798548080118,
  0.01099534126079007,
  0.011065354253198273,
  0.011135840888689201,
  0.01120680455598781,
  0.011278248669363332,
  0.011350176668825136,
  0.011422592020336715,
  0.011495498216010723,
  0.011568898774321613,
  0.011642797240316288,
  0.011717197185819539,
  0.01179210220964867,
  0.011867515937829991,
  0.011943442023812296,
  0.012019884148686108,
  0.012096846021399576,
  0.012174331378987182,
  0.012252343986782798,
  0.0123308876386578,
  0.0124099661572343,
  0.012489583394122214,
  0.012569743230150369,
  0.012650449575594675,
  0.01273170637041702,
  0.012813517584499067,
  0.012895887217881475,
  0.01297881930100614,
  0.013062317894955924,
...
pflatnorm2: Array[Double] = Array(
  0.010320534052761331,
  0.01